# Exploration du site Guts of Darkness

🎯 Objectif : 
Explorer la structure HTML d'une page d'album pour en extraire :
- Le nom de l'artiste
- Le nom de l'album
- Le texte de la critique
- Les styles (tags)
- Les notes (si disponible)

Les résultats de ce notebook serviront à construire le script `src/ingestion/guts_scraper.py`.


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import random
import re
import os


HEADERS = {
    "User-Agent": "MusicRecommenderBot/0.1 (+mailto:ton.email@example.com)"
}


In [5]:
url = "https://www.gutsofdarkness.com/god/objet.php?objet=12345"  # ← à remplacer par une vraie URL
r = requests.get(url, headers=HEADERS)
print("Statut :", r.status_code)
html = r.text[:10000]  # aperçu du début
print(html)


Statut : 200
<!DOCTYPE html>
<html lang="fr">
<head>
    <title>Guts Of Darkness &#8250; Foetus  &#8250;  Bedrock</title>
    <meta charset="UTF-8" />
    <meta http-equiv="Content-Language" content="fr-fr" />
    <meta name="viewport" content="width=device-width"/>
    <meta name="robots" content="index, follow" />
    <meta name="description" content="Chronique de Bedrock de Foetus" />
    <meta name="keywords" content="Guts Of Darkness, Guts, Of, Darkness, webzine, musique, musiques, chronique, chroniques, concert, concerts, groupe, groupes, artiste, artistes, Foetus  &#8250;  Bedrock" />
    <meta name="author" content="Guts Of Darkness"/>
    <meta name="Copyright" content="Guts Of Darkness" />
<!--    <base href="http://www.gutsofdarkness.com/god/" />  -->

    
    <link rel="stylesheet" type="text/css" media="screen" title="2021.flex" href="_global/_css/2021.flex/min-screen.css?v20250928" />
    <link rel="stylesheet" type="text/css" media="print" title="2020p" href="_global/_c

In [8]:
def parse_album(html: str, url: str) -> dict:
    soup = BeautifulSoup(html, "html.parser")

    # === TITRE & ARTISTE ===
    h1 = soup.select_one("h1")
    title = h1.find("em").get_text(strip=True) if h1 and h1.find("em") else None
    artist = h1.get_text(" ", strip=True).replace(title, "").replace(">", "").strip() if h1 else None

    # Alternative : line-up
    lineup = soup.select_one("#objetLineup p")
    if lineup and not artist:
        artist = lineup.get_text(" ", strip=True)

    # === INFORMATIONS ===
    info_div = soup.select_one("#objet-informations")
    infos = []
    if info_div:
        infos = [p.get_text(" ", strip=True) for p in info_div.find_all("p")]

    # === CHRONIQUE ===
    chronique_div = soup.select_one("div.objet-chronique")
    chronique_text = ""
    if chronique_div:
        ps = chronique_div.find_all("p")
        chronique_text = " ".join(p.get_text(" ", strip=True) for p in ps)
        chronique_text = re.sub(r"\s+", " ", chronique_text).strip()

    # === STYLES ===
    style_div = soup.select_one("div.objet-style")
    styles = [a.get_text(strip=True) for a in style_div.find_all("a")] if style_div else []

    # === NOTE DE LA CHRONIQUE ===
    sous_chronique_div = soup.select_one("div.objet-sous-chronique div.discrete-info")
    note_chronique = None
    if sous_chronique_div:
        pleines = len(sous_chronique_div.select("span.gfxNotePleine"))
        demi = len(sous_chronique_div.select("span.gfxNoteDemi"))
        vide = len(sous_chronique_div.select("span.gfxNoteVide"))
        note_chronique = pleines + 0.5 * demi

    # === ALBUMS "DANS LE MÊME ESPRIT" ===
    related_section = soup.select("div.mosaique a h1 em")
    same_spirit = [em.get_text(strip=True) for em in related_section] if related_section else []

    # === NOTE MOYENNE ===
    vote_div = soup.select_one("div#objetVote")
    note_moyenne = None
    if vote_div:
        pleines = len(vote_div.select("span.gfxNotePleine"))
        demi = len(vote_div.select("span.gfxNoteDemi"))
        note_moyenne = pleines + 0.5 * demi

    # === TAGS (ajoutés par utilisateurs) ===
    tags_div = soup.select_one("div#contenuObjetTags")
    tags_text = ""
    if tags_div:
        tags_text = tags_div.get_text(" ", strip=True)

    return {
        "album_name": title,
        "artist_name": artist,
        "lineup": lineup.get_text(" ", strip=True) if lineup else None,
        "informations": " ".join(infos),
        "chronique": chronique_text,
        "styles": ";".join(styles),
        "note_chronique": note_chronique,
        "note_moyenne": note_moyenne,
        "same_spirit": ";".join(same_spirit),
        "tags_text": tags_text,
        "source_url": url,
    }

In [13]:
data = parse_album(r.text, url)

for k, v in data.items():
    print(f"{k}: {v[:10000] if isinstance(v, str) else v}")

album_name: Bedrock
artist_name: Foetus  ›
lineup: J. G. Thirlwell (aka Clint Ruin)
informations: Enregistré à Milo, Paradise et Stroud Green, Londres. – 
Ingés-son : Charles Gray & Wayne Livesey – Mixé à Livingston, Londres Sorti sous le nom “The Foetus All-Nude Revue” - 
« GIVE ME A HOME WHERE THE DINOSAURS ROAM » - artwork par JG Thirwell
chronique: Encore un superbe maxi de Fœtus, que l’on retrouve – fort heureusement – aussi sur la compil de singles ‘Sink’ qui fait sans aucun doute partie des très grands moments de la carrière ce cinglé de J.G. Thirwell… La « Face A » (en français dans le texte, ambiance moulin rouge cyberpunk oblige) accueille donc l’un des plus grands titres de Fœtus (ici tapageusement annoncé comme un show de strip-tease), le bien nommé Bedrock, grandiose monstruosité qui pourrait se décrire par la greffe du larynx explosé de Tom Waits sur le corps de Ogre de Skinny Puppy opérée par le Docteur Mengele dans une cuve à uranium enrichi, pendant que Raspoutine joue

In [6]:
def generate_album_links(start_id: int, end_id: int):
    """
    Génère une liste d'URLs d'albums à partir des IDs numériques.
    Exemple : start_id=24000, end_id=24100 → 100 liens
    """
    BASE_URL = "https://www.gutsofdarkness.com/god/objet.php?objet="
    urls = [f"{BASE_URL}{i}" for i in range(start_id, end_id + 1)]
    print(f" {len(urls)} liens générés ({urls[0]} → {urls[-1]})")
    return urls

In [ ]:
# === Étape 5 : test sur plusieurs albums ===

urls = generate_album_links(00000, 25000)

output_path = "../data/processed/sample_albums.csv"

if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    scraped_urls = set(df_existing["source_url"])
    print(f" {len(scraped_urls)} albums déjà présents — ils seront ignorés.")
else:
    scraped_urls = set()
    print(" Aucun dataset existant, scraping complet.")

rows = []

for i, url in enumerate(urls, 1):
    if url in scraped_urls:
        print(" Déjà présent, ignoré.")
        continue
    print(f"\n[{i}/{len(urls)}] Scraping {url} ...")
    try:
        r = requests.get(url, headers=HEADERS, timeout=10)
        if r.status_code != 200:
            print(f" Erreur {r.status_code} sur {url}")
            continue

        parsed = parse_album(r.text, url)
        if parsed["album_name"] and parsed["artist_name"]:
            rows.append(parsed)
            print(f" {parsed['artist_name']} - {parsed['album_name']} récupéré")
        else:
            print(f" Informations incomplètes, album ignoré.")

    except Exception as e:
        print(f" Erreur sur {url} : {e}")

    sleep(1 + random.random() * 2)  # éviter d'enchaîner trop vite

df = pd.DataFrame(rows)
print("\n=== Aperçu des données ===")
display(df.head())

if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    df = pd.concat([df_existing, df], ignore_index=True)

# Nettoyage avant export
df.drop_duplicates(subset=["source_url"], inplace=True)
df.reset_index(drop=True, inplace=True)

df.to_csv(output_path, index=False, encoding="utf-8")
print(f"\n Données exportées vers {output_path} ({len(df)} albums uniques)")


 5001 liens générés (https://www.gutsofdarkness.com/god/objet.php?objet=20000 → https://www.gutsofdarkness.com/god/objet.php?objet=25000)
 79 albums déjà présents — ils seront ignorés.

[1/5001] Scraping https://www.gutsofdarkness.com/god/objet.php?objet=20000 ...
 Burzum  › - From the depths of darkness récupéré

[2/5001] Scraping https://www.gutsofdarkness.com/god/objet.php?objet=20001 ...
 Burzum  › - Sôl austan, Mâni vestan récupéré

[3/5001] Scraping https://www.gutsofdarkness.com/god/objet.php?objet=20002 ...
 Paradise Lost  › - In Requiem récupéré

[4/5001] Scraping https://www.gutsofdarkness.com/god/objet.php?objet=20003 ...
 Sextile  › - A thousand hands récupéré

[5/5001] Scraping https://www.gutsofdarkness.com/god/objet.php?objet=20004 ...
 Goatsblood  › - Drull récupéré

[6/5001] Scraping https://www.gutsofdarkness.com/god/objet.php?objet=20005 ...
 Depeche Mode  › - Spirit récupéré

[7/5001] Scraping https://www.gutsofdarkness.com/god/objet.php?objet=20006 ...
 Lhasa  › - 

KeyboardInterrupt: 